In [ ]:
import os
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
import torch
# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    print("cuda Available")

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
batch_size = 64
num_workers = 0
loaders_scratch = {}

normalize = transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
dataTransform = transforms.Compose([transforms.Resize(250),transforms.CenterCrop(224), transforms.ToTensor(),normalize])
#dataTransform2 =  transforms.Compose([transforms.Resize(224),transforms.ToTensor(), normalize])

train_data = datasets.ImageFolder(root='/data/dog_images/train', transform=dataTransform)
test_data = datasets.ImageFolder(root='/data/dog_images/test', transform=dataTransform)
eval_data = datasets.ImageFolder(root='/data/dog_images/valid', transform=dataTransform)

loaders_scratch['train'] = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers=num_workers)
loaders_scratch['test'] = torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers=num_workers)
loaders_scratch['valid'] = torch.utils.data.DataLoader(eval_data, batch_size = batch_size, num_workers=num_workers)
    
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        #input layer [224,224,3]
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        #Maxpool Layer -> [64,112,112]
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        #Maxpool Layer -> [128,56,56]
        #self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        #self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        #Maxpool Layer -> [256,28,28]
       # self.conv7 = nn.Conv2d(256,512, 3, padding=1)
        #Maxpool Layer -> [512,14,14]
        
        #Linear Layers
        self.fc1 = nn.Linear(25088, 5000)
        self.fc2 = nn.Linear(5000, 133)
        #self.fc3 = nn.Linear(1000 , 133)
        #self.fc3 = nn.Linear(112,133)
        
        self.dropout = nn.Dropout(p=0.5)
        self.pool = nn.MaxPool2d(2,2)
    
    def forward(self, x):
        ## Define forward behavior
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        #x = F.relu(self.conv5(x))
        #x = F.relu(self.conv6(x))
        #x = self.pool(x)
        #x = F.relu(self.conv7(x))
        #x = self.pool(x)
        x = x.view(-1, 25088)   
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        #x = F.relu(self.fc2(x))
        #x = self.dropout(x)        
        #x = F.log_softmax(self.fc3(x), dim=1)
        x = self.fc2(x)
        return x

#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()
    
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(),lr=0.01)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        ttloss = 0.0
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        print_count = 0
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model.forward(data)
            
            loss = criterion(output,target)
            loss.backward()
            
            optimizer.step()                
               
                
            train_loss = ttloss + ((1 / (batch_idx + 1)) * (loss.data - ttloss))
            print_count += 1
            if (batch_idx % 32 == 0):
                print('   Batch: {} \tEpoch: {} \tTraining Loss: {:.6f} '.format(batch_idx, epoch, train_loss))
                print_count = 0
           
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
                
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model.forward(data)
            loss = criterion(output,target)
            
            valid_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if(valid_loss < valid_loss_min):
            valid_loss_min = valid_loss
            print("Model Saved")
            torch.save(model.state_dict(), 'model_scratch.pt')
    # return trained model
    return model


# train the model
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

